In [ ]:
import tensorflow as tf
import numpy as np


In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
pip install -q keras

In [ ]:
import keras

In [ ]:
base_dir =  '/content/drive/MyDrive/MnagoLeaf1/BaseDirectoryForCNNModel'

#Image Preprocessing

In [ ]:
IMAZE_SIZE = 224
BATCH_SIZE = 16


train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
     zoom_range=0.2,
     rotation_range =40,
     height_shift_range =0.2,
     width_shift_range=0.2,
     brightness_range=[0.2,1.0],
     channel_shift_range=0.0,
    horizontal_flip=True,
    vertical_flip=True,
    fill_mode='nearest',
    validation_split=0.3)




train_set =train_datagen.flow_from_directory(
    base_dir,
    target_size = (224,224),
    batch_size = BATCH_SIZE,
    subset = 'training',
     class_mode='categorical'
  
   
     
)


test_datagen =  tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255,
    validation_split=0.3)



test_set = test_datagen.flow_from_directory(
    base_dir,
    target_size = (224,224),
    batch_size=BATCH_SIZE,
    subset = 'validation',
    class_mode='categorical'
)

Found 1655 images belonging to 4 classes.
Found 706 images belonging to 4 classes.


# CNN **MODEL**

In [ ]:
cnn = tf.keras.Sequential()
cnn.add(tf.keras.layers.Conv2D(filters=32,padding='same',strides=1,kernel_size=2,activation='relu',input_shape=(224,224,3)))
cnn.add(tf.keras.layers.MaxPool2D(pool_size =2,strides  = 1))
cnn.add(tf.keras.layers.Conv2D(filters=64,padding='same',strides=1,kernel_size=2,activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size = 2,strides  = 1))
cnn.add(tf.keras.layers.Conv2D(filters=64,padding='same',strides=1,kernel_size=2,activation='relu'))
cnn.add(tf.keras.layers.Flatten())                      
cnn.add(tf.keras.layers.Dense(4,activation='softmax')) 



**Compile**


In [ ]:
cnn.compile(tf.keras.optimizers.Adam(),loss="categorical_crossentropy",metrics=['accuracy'])

In [ ]:

history=cnn.fit(train_set,epochs=20,steps_per_epoch=1655/16,validation_data=test_set,batch_size=BATCH_SIZE,validation_steps=706/16,verbose = 1)


Epoch 1/20
103/103 [==============================] - 75s 389ms/step - loss: 4.9266 - accuracy: 0.6205 - val_loss: 0.8280 - val_accuracy: 0.7833
Epoch 2/20
103/103 [==============================] - 39s 375ms/step - loss: 0.3801 - accuracy: 0.8979 - val_loss: 0.4687 - val_accuracy: 0.8739
Epoch 3/20
103/103 [==============================] - 39s 375ms/step - loss: 0.3361 - accuracy: 0.9033 - val_loss: 0.3623 - val_accuracy: 0.9136
Epoch 4/20
103/103 [==============================] - 39s 374ms/step - loss: 0.2518 - accuracy: 0.9148 - val_loss: 0.4704 - val_accuracy: 0.8966
Epoch 5/20
103/103 [==============================] - 39s 375ms/step - loss: 0.1797 - accuracy: 0.9462 - val_loss: 0.7135 - val_accuracy: 0.8598
Epoch 6/20
103/103 [==============================] - 39s 373ms/step - loss: 0.1083 - accuracy: 0.9722 - val_loss: 1.3012 - val_accuracy: 0.7663
Epoch 7/20
103/103 [==============================] - 39s 373ms/step - loss: 0.1315 - accuracy: 0.9631 - val_loss: 0.3923 - val_ac

# ***Prediction On Single Imagge***

In [ ]:
from keras.preprocessing import image
test_image = image.load_img('/content/drive/MyDrive/MnagoLeaf1/BaseDirectoryForCNNModel/Red Rust/20211011_132640 (Custom).jpg',target_size=(224,224),)
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)
result=cnn.predict(test_image)


In [ ]:
train_set.class_indices        

{'Die Back': 0, 'Powdery Mildew': 1, 'Red Rust': 2, 'Weevil': 3}

In [ ]:
print(result)

[[0. 0. 1. 0.]]


In [ ]:
if result[0][0]==1:
  print('Die Back')
elif result[0][1]==1:
  print('Powdery Mildew')
elif result[0][2]==1:
  print('Red Rust')
elif result[0][3]==1:  
  print('Weevil')

Red Rust


In [ ]:
train_loss,train_acc=cnn.evaluate(train_set)

104/104 [==============================] - 29s 276ms/step - loss: 0.0582 - accuracy: 0.9813


In [ ]:
validation_loss,val_acc =cnn.evaluate(test_set)

45/45 [==============================] - 4s 82ms/step - loss: 1.1527 - accuracy: 0.8499


In [ ]:
print('Train_Acc:%.3f, val_Acc:%.3f'%(train_acc,val_acc))

Train_Acc:0.981, val_Acc:0.850


In [ ]:
print('Train_loss:%.3f, validation_loss:%.3f'%(train_loss,validation_loss))

Train_loss:0.058, validation_loss:1.153


In [ ]:
history.history?

In [ ]:
history.history.keys()

dict_keys(['loss', 'accuracy', 'val_loss', 'val_accuracy'])

# **Train Accuracy**

In [ ]:
import matplotlib.pyplot as plt
plt.plot(history.history['accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train'],loc='upper left')
plt.show()

In [ ]:
import matplotlib.pyplot as plt 
plt.plot(history.history['loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train'],loc='upper left')
plt.show()

In [ ]:
train_loss=history.history['loss']
val_loss=history.history['val_loss']
train_acc=history.history['accuracy']
val_acc=history.history['val_accuracy']
xc=range(20)
print(xc)

# ***Train loss vs Validation loss***

In [ ]:

plt.plot(xc,train_loss)
plt.plot(xc,val_loss)
plt.xlabel('num of epoch')
plt.ylabel('loss')
plt.title('Train loss vs Validation loss')
plt.legend(['train','validation'])
plt.show()


# ***Train accuracy vs Validation accuracy***

---



In [ ]:

plt.plot(xc,train_acc)
plt.plot(xc,val_acc)
plt.xlabel('num of epoch')
plt.ylabel('accuracy')
plt.title('Train accuracy vs Validation accuracy')
plt.legend(['train','validation'], loc=4)
plt.show()


# **Precission,Recall,F1 score**

In [ ]:
pip install sklearn

In [ ]:
import sklearn

In [ ]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import plot_confusion_matrix
from sklearn.metrics import plot_precision_recall_curve
from sklearn.metrics import plot_roc_curve
from sklearn.metrics import precision_recall_fscore_support



In [ ]:
target_names = []

for key in train_set.class_indices:

    target_names.append(key)


print(target_names)

In [ ]:


Y_pred =cnn.predict(test_set)
y_pred = np.argmax(Y_pred,axis=1)
print('Confusion Matrix')
cm=confusion_matrix(test_set.classes,y_pred)
print(cm)
plt.matshow(cm)


In [ ]:
print(classification_report(test_set.classes, y_pred, target_names=target_names))

                precision    recall  f1-score   support

      Die Back       0.23      0.23      0.23       153
Powdery Mildew       0.08      0.03      0.04        66
      Red Rust       0.30      0.27      0.28       220
        Weevil       0.37      0.45      0.41       267

      accuracy                           0.31       706
     macro avg       0.24      0.25      0.24       706
  weighted avg       0.29      0.31      0.30       706



In [ ]:
figure=plt.figure(figsize=(8,8))
